In [ ]:
%%capture
!pip install datasets==2.6.1
!pip install install transformers==4.28
!pip install huggingface_hub
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [ ]:
import torch
import torchaudio
import datasets
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    batch["sentence"] = batch["sentence"].replace('！ ', '')
    batch["sentence"] = batch["sentence"].replace('，', '')
    batch["sentence"] = batch["sentence"].replace('é', 'e')    
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    resampler = torchaudio.transforms.Resample(sampling_rate, 16_000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

In [ ]:
# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = processor.batch_decode(pred_ids)
    return batch

In [ ]:
test_dataset = load_dataset("common_voice", "id", split="test")
wer = load_metric("wer")

/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


Generating train split:   0%|          | 0/2130 [00:00<?, ? examples/s]

/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


Generating test split:   0%|          | 0/1844 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1835 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/6782 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/8696 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/470 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/id/6.1.0/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf. Subsequent calls will reuse this data.


<ipython-input-7-651ee0842fa0>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer = load_metric("wer")


In [ ]:
%%capture

model_name = "mfaiq2307/faiq-wav2vec2-xls-r-300m-indo-demo-t4"
#model_name = "./wav2vec2-large-xlsr-indonesian-articial-CV"
#model_name = "cahya/wav2vec2-large-xlsr-indonesian"
#model_name = "ayameRushia/wav2vec2-large-xlsr-indonesia-demo"
#model_name = "Galuh/wav2vec2-large-xlsr-indonesian-demo"

processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name) 
model.to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\'\”]'
test_dataset = test_dataset.map(speech_file_to_array_fn)

  0%|          | 0/1844 [00:00<?, ?ex/s]

In [ ]:
result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))

  0%|          | 0/231 [00:00<?, ?ba/s]

WER: 26.393809
